In [1]:
%pylab inline
import pandas as pd
import numpy as np
import os
import sklearn as sk
from sklearn.cluster import AffinityPropagation
from sklearn import datasets
from sklearn import metrics
from multiprocessing import Pool
import pickle

Populating the interactive namespace from numpy and matplotlib


In [31]:
thca = pd.read_csv('thca_maf_no_head.txt', sep='\t')

In [37]:
thca.columns
thca[['Reference_Allele', 'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2']].head(100)
print "super weird that there are no bi-allelic mutations..."

super weird that there are no bi-allelic mutations...


In [26]:
def check_bi_allelic(x):
    if x[0] == x[1]:
        return True
    else:
        return False
thca["Bi_Allelic"] = thca[['Tumor_Seq_Allele1', 'Tumor_Seq_Allele2']].apply(check_bi_allelic, axis=1)
thca['Patient_Barcode'] = thca.Tumor_Sample_Barcode.str[:12]

In [27]:
columns = ["Hugo_Symbol", "Entrez_Gene_Id", "Chromosome", "Start_position", "Variant_Classification", "Bi_Allelic", "Patient_Barcode", "Protein_Change"]
thca = thca[columns]

In [28]:
thca.head()

,Hugo_Symbol,Entrez_Gene_Id,Chromosome,Start_position,Variant_Classification,Bi_Allelic,Patient_Barcode,Protein_Change
0,ZNF677,342926,19,53740670,Missense_Mutation,False,TCGA-CE-A27D,p.G437D
1,MCMBP,79892,10,121612707,Splice_Site,False,TCGA-CE-A27D,p.A144_splice
2,CYP2U1,113612,4,108866315,Missense_Mutation,False,TCGA-CE-A27D,p.A227V
3,CCDC89,220388,11,85396623,Missense_Mutation,False,TCGA-CE-A27D,p.E184G
4,ANKRD17,26057,4,73957562,Missense_Mutation,False,TCGA-CE-A27D,p.P1928L


In [30]:
thca.Bi_Allelic.value_counts()

False    7458
dtype: int64

In [19]:
len(thca.Tumor_Sample_Barcode.unique())

405

In [40]:
def combine(x):
    gene = x[0]
    loc = x[1]
    return str(gene) + "_" + str(loc)
thca['SNP'] = thca[['Entrez_Gene_Id', 'Start_position']].apply(combine, axis=1)
thca.to_csv('thca.csv')

In [41]:
muts = pd.DataFrame(thca.SNP.value_counts())
muts = muts.reset_index()
muts.columns = ['SNP', 'counts']
muts.head()

,SNP,counts
0,673_140453136,248
1,4893_115256529,31
2,3265_533874,12
3,4893_115256530,7
4,54726_146059006,6


In [42]:
print "It might make more sense to just create a SNP/Protein change table and add the protein changes to the old table?"

It might make more sense to just create a SNP/Protein change table and add the protein changes to the old table?


In [45]:
pc = thca[['SNP', 'Protein_Change']]
print pc.count()
print pc.drop_duplicates().count()

SNP               7458
Protein_Change    7372
dtype: int64
SNP               6755
Protein_Change    6672
dtype: int64


In [46]:
pc.to_csv('pc.csv')